<a href="https://colab.research.google.com/github/satyamanikantareddychintapalli/FFML_LAB_23B21A4555/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

Yes, averaging the validation accuracy across multiple splits generally provides more consistent and reliable results compared to relying on a single split. This approach, known as cross-validation, helps to mitigate the variability that can occur due to the specific choice of the training and validation sets.

### Why Averaging Across Multiple Splits is More Consistent:
1. **Reduces Variance**: The accuracy obtained from a single split can be highly dependent on the particular samples that are included in the training and validation sets. By averaging the accuracy across multiple splits, you smooth out the variability that comes from different data distributions in these splits.

2. **Better Generalization Estimate**: Each split in cross-validation uses a different subset of data for validation, so averaging the results gives a more reliable estimate of how the model is expected to perform on unseen data. This is particularly useful when the dataset is small, as a single split may not be representative of the entire data distribution.

3. **More Robust Evaluation**: Averaging over multiple splits reduces the risk of overestimating or underestimating the model's performance. It provides a more comprehensive evaluation by ensuring that the model's accuracy is not overly influenced by outliers or specific data points that could skew the results in a single split.

4. **Cross-Validation**: In more advanced forms of cross-validation, such as k-fold cross-validation, the entire dataset is divided into k subsets, and the model is trained and validated k times, each time using a different subset as the validation set and the remaining k-1 subsets as the training set. The final accuracy is the average of the k results, providing a stable and generalized performance metric.

### Example:
If you perform 10 different splits and calculate the validation accuracy for each split, the individual accuracies might vary, but the average of these 10 accuracies will give you a more stable estimate. This averaged accuracy is less sensitive to the specific configuration of each individual split.

In summary, averaging validation accuracy across multiple splits does indeed give more consistent and reliable results, which is why it's a common practice in machine learning model evaluation.



Averaging validation accuracy across multiple splits generally provides a more accurate estimate of the test accuracy, but with some caveats. Here's how it works:

### How Averaging Validation Accuracy Improves Test Accuracy Estimation:

1. **Reduces Overfitting to a Specific Validation Set**: When you rely on a single train-validation split, the model might perform well or poorly on that particular validation set due to random variations in the data. By averaging over multiple splits, you reduce the influence of any particular set of validation data, leading to a more balanced and accurate estimate of how the model will perform on unseen test data.

2. **Improves Generalization**: Averaging validation accuracy over multiple splits tends to provide a better approximation of how the model will generalize to completely unseen data (i.e., the test set). This is because the model is evaluated on different subsets of data each time, which helps capture the overall performance across the entire dataset.

3. **Mitigates the Impact of Variability**: Single-split validation can lead to high variability in performance metrics if the validation set is not representative of the overall data distribution. Multiple splits, and the subsequent averaging, help smooth out these fluctuations, resulting in an accuracy estimate that is closer to what you would expect on the test set.

### Caveats and Considerations:

1. **Bias-Variance Tradeoff**: While averaging reduces variance in the accuracy estimate, it doesn’t completely eliminate bias. If your validation sets are not representative of the test set (e.g., if they are drawn from a slightly different distribution), the estimate could still be off.

2. **Size of the Dataset**: The accuracy of the estimate also depends on the size of the dataset. If the dataset is very small, even cross-validation may not perfectly represent the test set. However, for most reasonably sized datasets, cross-validation provides a good approximation.

3. **Model Complexity**: If your model is very complex and prone to overfitting, even averaging across multiple validation splits might still yield overly optimistic estimates of test accuracy. In such cases, more sophisticated techniques, like nested cross-validation, might be needed.

### Conclusion:
Averaging validation accuracy across multiple splits typically gives a more accurate and reliable estimate of test accuracy compared to using a single split. This approach helps ensure that your model’s performance metric is closer to what you can expect on unseen data, making it a fundamental practice in machine learning model evaluation. However, the accuracy of the estimate can still be influenced by factors like data representativeness, model complexity, and dataset size.



The number of iterations (i.e., the number of different train-validation splits used in cross-validation) plays a significant role in determining the quality and stability of the estimate of the test accuracy. Here's how the number of iterations affects the estimate:

### Effect of Number of Iterations on the Estimate

1. **Stability and Consistency**:
   - **Higher Iterations**: As the number of iterations increases, the estimate of the test accuracy becomes more stable and consistent. This is because averaging over more iterations reduces the influence of any one particular split, leading to a smoother and more reliable estimate.
   - **Lower Iterations**: With fewer iterations, the estimate may be more susceptible to variability. The result might be more influenced by specific splits, leading to a less reliable and potentially more volatile estimate.

2. **Convergence of the Estimate**:
   - With a higher number of iterations, the estimate of the test accuracy tends to converge to a more accurate value. Essentially, as you increase the number of iterations, you are sampling more variations of the train-validation split, which gives a better approximation of the model's performance across the entire dataset.
   - After a certain point, additional iterations contribute less to the accuracy of the estimate, as the results start to converge. This means there’s diminishing returns with very high iterations, but increasing iterations up to this convergence point does generally lead to a better estimate.

3. **Reduced Impact of Outliers**:
   - Outliers or unusual data points can have a significant impact on accuracy if only a few iterations are used. As the number of iterations increases, the influence of these outliers diminishes, leading to a more robust and accurate estimate.

4. **Computational Cost**:
   - While more iterations can lead to a better estimate, they also increase the computational cost. This is an important tradeoff to consider, especially for complex models or large datasets. You need to balance the desire for accuracy with the available computational resources and time.

### Practical Implications:

- **Few Iterations**: If you only perform a small number of iterations (e.g., 3-5), the estimate might be less reliable and could fluctuate depending on the specific splits used.
- **Moderate Iterations (e.g., 10-20)**: This range is often sufficient for many practical purposes, offering a good balance between computational cost and the stability of the estimate.
- **High Iterations (e.g., 50-100 or more)**: This can lead to very stable and accurate estimates but at a higher computational cost. The improvement in estimate accuracy beyond a certain number of iterations may be marginal.

### Conclusion:
Increasing the number of iterations generally leads to a more accurate and reliable estimate of test accuracy, as it reduces variability and allows the estimate to converge to a more stable value. However, beyond a certain point, the benefits of additional iterations diminish, and it becomes a tradeoff between computational cost and the desired accuracy of the estimate. For most cases, a moderate number of iterations provides a good balance.


Increasing the number of iterations in cross-validation can help mitigate some issues associated with small training or validation datasets, but it doesn't fully solve the underlying problems. Here’s how it affects small datasets and the associated challenges:

### Dealing with a Very Small Training Dataset

1. **Challenges**:
   - **Underfitting**: A very small training dataset may not provide enough information for the model to learn effectively, leading to underfitting. The model might not capture the underlying patterns in the data, resulting in poor performance on both the training and validation sets.
   - **High Variance**: Small training datasets can cause high variance in model performance across different splits, leading to less reliable estimates of accuracy.

2. **Effect of Increasing Iterations**:
   - **More Reliable Estimate**: By increasing the number of iterations, you can get a more reliable estimate of how well the model performs given the limited data. This is because you’re averaging the model’s performance across many different splits, which can help smooth out the variance.
   - **Doesn't Improve Learning**: However, increasing iterations does not fundamentally improve the model’s ability to learn from the small dataset. The model is still constrained by the limited amount of data, which means it might not generalize well to unseen data, regardless of how many splits you use.

3. **Potential Solutions**:
   - **Data Augmentation**: In some cases, you might use techniques like data augmentation (especially in fields like image processing) to artificially increase the size of the training set.
   - **Transfer Learning**: Leveraging pre-trained models or transfer learning can help when the training data is limited, especially in fields like NLP or computer vision.
   - **Simpler Models**: Using simpler models that require fewer data to learn effectively might be a better approach when dealing with very small training datasets.

### Dealing with a Very Small Validation Dataset

1. **Challenges**:
   - **High Variability in Validation Accuracy**: With a small validation dataset, the accuracy estimates can vary widely depending on the specific examples included in the validation set. This can lead to unreliable estimates of how well the model will perform on unseen data.

2. **Effect of Increasing Iterations**:
   - **Reduced Variance in Estimates**: Increasing the number of iterations helps to reduce the variability in validation accuracy. By averaging over many different validation sets, you obtain a more stable and reliable estimate of model performance.
   - **Limited Representativeness**: While increasing iterations helps, the fundamental issue remains that a small validation set might not be representative of the broader dataset. The accuracy estimate could still be biased if the validation sets are not capturing the full diversity of the data.

3. **Potential Solutions**:
   - **Cross-Validation**: Techniques like k-fold cross-validation, where the data is split into k parts and each part is used as a validation set in turn, can help maximize the use of small datasets. This allows every data point to be used for both training and validation, making the most out of limited data.
   - **Stratified Sampling**: Ensuring that each validation set is representative of the overall class distribution (in classification tasks) can also improve the reliability of the accuracy estimate.

### Conclusion:
Increasing the number of iterations in cross-validation helps in obtaining more consistent and reliable estimates of model performance, especially when dealing with small validation datasets. However, it does not fundamentally address the limitations imposed by small training datasets. While it can reduce variance in performance estimates, the effectiveness of the model itself is still limited by the amount of data it has to learn from. To truly improve performance, alternative strategies like data augmentation, transfer learning, or using simpler models may be necessary.
